<a href="https://colab.research.google.com/github/sunny9sinha/TSAI_Session_6/blob/main/Session6_Assignment_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import pandas as pd
import numpy as np
import csv

!pip install googletrans==3.1.0a0

     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 1.0MB 18.0MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=9a9552f18daef9b8104039ed53bbe08ba75b5fecb7bb021fce21d5c9a932f224
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans


In [2]:
import random
import googletrans
from googletrans import Translator

def translate(sentence_translated):
  # translator = Translator(service_urls=['translate.googleapis.com'])
  translator = Translator()
  # sentence = ['']

  # available_langs = list(googletrans.LANGUAGES.keys()) 
  # trans_lang = random.choice(available_langs) 
  # print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

  sentence_translated = pd.DataFrame(sentence_translated.iloc[0:400,:]).reset_index().drop(columns=['index'])

  trans_array = np.array(sentence_translated['tweets'])
  # int(len(trans_array)/100)

  for i in range(400):
    translations = translator.translate(trans_array[i],src='en', dest='czech') 
    #print(translations)
    t_text = translations.text
    print('czech: ',t_text)

    translations_en_random = translator.translate(t_text, src='czech', dest='en') 
    en_text = translations_en_random.text
    print('Eng: ',en_text)
    trans_array[i] = en_text


  sentence_translated['tweets'] = trans_array
  return sentence_translated



In [3]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

In [4]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class encoders(nn.Module):

  def __init__(self,vocab_size, embedding_dim, hidden_dim, n_layers, dropout):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,embedding_dim)
    self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
    
  def forward(self,text,text_lengths):
    embedded = self.embedding(text)
    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
    packed_output, (hidden, cell) = self.encoder(packed_embedded)
    return hidden

class decoders(nn.Module):

  def __init__(self, input_to_decoder_size, decoder_hidden_size, no_times_decoder_cell_has_to_run):
    super().__init__()
    self.decoder_single_rnn_cell = nn.LSTMCell(input_to_decoder_size,decoder_hidden_size)
    self.no_times_decoder_cell_has_to_run = no_times_decoder_cell_has_to_run
    self.decoder_hidden_size = decoder_hidden_size

  def forward(self, encoder_context_vector):
    encoder_context_vector = encoder_context_vector.squeeze()
    hx = torch.zeros(encoder_context_vector.size(0),self.decoder_hidden_size).to(device)
    cx = torch.zeros(encoder_context_vector.size(0),self.decoder_hidden_size).to(device)
    for i in range(self.no_times_decoder_cell_has_to_run):
      hx,cx = self.decoder_single_rnn_cell(encoder_context_vector,(hx,cx))
    return hx

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, encoder, decoder, hidden_dim, output_dim):
        
      super().__init__()
      self.encoder = encoder
      self.decoder = decoder
      self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self,src,src_len):
      enc_hidden = self.encoder(src,src_len)
      #print('Encoder: ',enc_hidden)
      dec_hidden = self.decoder(enc_hidden)
      #print('Decoder: ', dec_hidden)
      dense_outputs = self.fc(dec_hidden)
      #print('----',dense_outputs[0])
      #output = F.softmax(dense_outputs,dim=1)
      #print('---',output)
      return dense_outputs

In [6]:
tweet_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tweets (1).csv")
tweet_data.head()

,tweets,labels
0,Obama has called the GOP budget social Darwini...,1
1,"In his teen years, Obama has been known to use...",0
2,IPA Congratulates President Barack Obama for L...,0
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0
4,RT @wardollarshome: Obama has approved more ta...,1


In [7]:
tweet_data.shape

(1364, 2)

In [8]:
tweet_data.labels.value_counts()

0    931
1    352
2     81
Name: labels, dtype: int64

In [9]:
def delete(sent_delete):
  del_array = np.array(sent_delete['tweets'])
  for i in range(len(del_array)):
    words = str(del_array[i]).split()
    word_selected = random.choice(words)
    word_deleted = random_deletion(word_selected, 0.4)
    del_word = ''
    for k in word_deleted:
      del_word+=k
    del_array[i] = str(del_array[i]).replace(word_selected,del_word)
  sent_delete['tweets'] = del_array
  return sent_delete

tweet_delete = pd.DataFrame(tweet_data.iloc[:,:]).reset_index().drop(columns=['index'])
print(tweet_delete.head())
tweet_delete = delete(tweet_delete)

                                              tweets  labels
0  Obama has called the GOP budget social Darwini...       1
1  In his teen years, Obama has been known to use...       0
2  IPA Congratulates President Barack Obama for L...       0
3  RT @Professor_Why: #WhatsRomneyHiding - his co...       0
4  RT @wardollarshome: Obama has approved more ta...       1


In [10]:
print(tweet_delete.head(10))

                                              tweets  labels
0  Obama has called the GOP budget oc Darwinism. ...       1
1  In his teen years, Obama has been kwn to use m...       0
2  IPA ontules President Barack Obama for Leaders...       0
3  RT @Professor_Why: #WhatsRomneyHiding - his co...       0
4  RT @wardollarshome: Obama has approved more ta...       1
5  Video shows federal officials joking about cos...       0
6  one Chicago kid who sa "Obama is my man" tells...       0
7  RT @ohgirlphrase: American kid "You're from th...       0
8  A valid explanation for why ama won't let wome...       1
9  President Obama &lt; Lindsay Loh RUMORS beginn...       0


In [11]:
tweet_swap = pd.DataFrame(tweet_data.iloc[:,:]).reset_index().drop(columns=['index']) 
print(tweet_swap.head())

                                              tweets  labels
0  Obama has called the GOP budget social Darwini...       1
1  In his teen years, Obama has been known to use...       0
2  IPA Congratulates President Barack Obama for L...       0
3  RT @Professor_Why: #WhatsRomneyHiding - his co...       0
4  RT @wardollarshome: Obama has approved more ta...       1


In [12]:
def swap(tweet_swap):
  for i in tweet_swap.index:
    words = tweet_swap['tweets'][i].split()
    if len(words) <=1:
      continue
    swapped_array = random_swap(words)
    tweet_swap['tweets'][i] = ' '.join([str(elem) for elem in swapped_array])
  return tweet_swap

In [13]:
tweet_swap = swap(tweet_swap)
tweet_swap.head(6)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,tweets,labels
0,"try, has called the budget Nice in Darwinism. ...",1
1,"cocaine. known marijuana years, Obama has use ...",0
2,"05, Congratulates IPA Regarding (BUSINESS for ...",0
3,not was #WhatsRomneyHiding - his connection to...,0
4,RT US Obama has http://t.co/bfC4gbBW prez; tar...,1
5,cost lavish federal officials joking about Vid...,0


In [14]:
print(tweet_data.shape)
print(tweet_delete.shape)
print(tweet_swap.shape)

(1364, 2)
(1364, 2)
(1364, 2)


In [15]:
tweet_translated = tweet_data
tweet_translated = translate(tweet_translated)
print(tweet_translated.shape)

czech:  Obama nazval rozpočet GOP sociálním darwinismem. Pěkný pokus, ale věří v sociální kreacionismus.
Eng:  Obama called the GOP budget social Darwinism. Nice try, but he believes in social creationism.
czech:  V jeho dospívání bylo známo, že Obama užíval marihuanu a kokain.
Eng:  In his teens, Obama was known to use marijuana and cocaine.
czech:  IPA blahopřeje prezidentu Baracku Obamovi k vedení ohledně zákona o pracovních příležitostech: WASHINGTON, 5. dubna 2012 (OBCHODNÍ ... http://t.co/8le3DC8E
Eng:  IPA congratulates President Barack Obama on leadership on Job Opportunity Act: WASHINGTON, April 5, 2012 (BUSINESS ... http://t.co/8le3DC8E
czech:  RT @Professor_Why: #WhatsRomneyHiding - jeho spojení s příznivci teorie kritické rasy .... No počkej, to byl Obama, ne Romney ...
Eng:  RT @Professor_Why: #WhatsRomneyHiding - his connection with supporters of critical race theory .... Well wait, it was Obama, not Romney ...
czech:  RT @wardollarshome: Obama schválil cílenější atentáty

In [16]:
tweet_data_1 = pd.concat([tweet_data,tweet_swap,tweet_delete,tweet_translated]).reset_index().drop(columns=['index'])
tweet_data_1.head()

,tweets,labels
0,Obama has called the GOP budget social Darwini...,1
1,"In his teen years, Obama has been known to use...",0
2,IPA Congratulates President Barack Obama for L...,0
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0
4,RT @wardollarshome: Obama has approved more ta...,1


In [17]:
tweet_data_1.labels.value_counts()

0    3065
1    1165
2     262
Name: labels, dtype: int64

In [18]:
new_tweet_delete_1 = tweet_data_1[tweet_data_1.labels==1].reset_index().drop(columns=['index'])
new_tweet_delete_2 = tweet_data_1[tweet_data_1.labels==2].reset_index().drop(columns=['index'])
new_tweet_delete = pd.concat([new_tweet_delete_1,new_tweet_delete_2]).reset_index().drop(columns=['index'])
new_tweet_delete.head()

,tweets,labels
0,Obama has called the GOP budget social Darwini...,1
1,RT @wardollarshome: Obama has approved more ta...,1
2,A valid explanation for why Obama won't let wo...,1
3,Obama's Gender Advantage Extends to the States...,1
4,Here's How Obama and the Democrats Will Win in...,1


In [19]:
new_tweet_delete = delete(new_tweet_delete)
new_tweet_delete = swap(new_tweet_delete)
new_tweet_delete.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,tweets,labels
0,called has Obama they believe g GOP social Nic...,1
1,has @wardollarshome: Obama t approved more tar...,1
2,Obama the A o why women won't let explanation ...,1
3,Obama's State Extends Advantage to the Gender ...,1
4,polls ow and Obama the Democrats Here's Let's ...,1


In [20]:
tweet_data = pd.concat([tweet_data_1,new_tweet_delete]).reset_index().drop(columns=['index'])
print(tweet_data.head())
print(tweet_data.shape)
print(tweet_data.labels.value_counts())

                                              tweets  labels
0  Obama has called the GOP budget social Darwini...       1
1  In his teen years, Obama has been known to use...       0
2  IPA Congratulates President Barack Obama for L...       0
3  RT @Professor_Why: #WhatsRomneyHiding - his co...       0
4  RT @wardollarshome: Obama has approved more ta...       1
(5919, 2)
0    3065
1    2330
2     524
Name: labels, dtype: int64


In [21]:
import random
import torch, torchtext
from torchtext import data
SEED = 43
torch.manual_seed(SEED)

In [22]:
Tweet = torchtext.legacy.data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = torchtext.legacy.data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [23]:
fields = [('tweet', Tweet), ('label', Label)]

In [24]:
example = [torchtext.legacy.data.Example.fromlist([tweet_data.tweets[i],tweet_data.labels[i]], fields) for i in range(tweet_data.shape[0])] 

In [25]:
twitterDataset = torchtext.legacy.data.Dataset(example, fields)

In [26]:
(train, valid) = twitterDataset.split(split_ratio=[85, 15], random_state = random.seed(SEED))

In [27]:
len(train), len(valid)

(5031, 888)

In [28]:
vars(twitterDataset.examples[0])

{'label': 1,
 'tweet': ['Obama',
  'has',
  'called',
  'the',
  'GOP',
  'budget',
  'social',
  'Darwinism',
  '.',
  'Nice',
  'try',
  ',',
  'but',
  'they',
  'believe',
  'in',
  'social',
  'creationism',
  '.']}

In [29]:
Tweet.build_vocab(train)
Label.build_vocab(train)

In [30]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  6511
Size of label vocab :  3
Top 10 words appreared repeatedly : [('Obama', 4455), ('.', 3453), (':', 3388), ('#', 3148), (',', 2530), ('the', 2300), ('"', 2289), ('RT', 2243), ('to', 1663), ('?', 1564)]
Labels :  defaultdict(None, {0: 0, 1: 1, 2: 2})


In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [32]:
train_iterator, valid_iterator = torchtext.legacy.data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.tweet),
                                                            sort_within_batch=True, device = device)

In [33]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

In [34]:
# Define hyperparameters
size_of_vocab = len(Tweet.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 3
num_layers = 1
dropout = 0.2

# Instantiate the model
enc = encoders(size_of_vocab, embedding_dim, num_hidden_nodes, num_layers, dropout = dropout)
dec = decoders(num_hidden_nodes,num_hidden_nodes,10)

model = classifier(enc,dec,num_hidden_nodes,num_output_nodes).to(device)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [35]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (encoder): encoders(
    (embedding): Embedding(6511, 300)
    (encoder): LSTM(300, 100, batch_first=True, dropout=0.2)
  )
  (decoder): decoders(
    (decoder_single_rnn_cell): LSTMCell(100, 100)
  )
  (fc): Linear(in_features=100, out_features=3, bias=True)
)
The model has 2,195,203 trainable parameters


In [36]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [37]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweet  
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.label)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [38]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweet
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [39]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 0.927 | Train Acc: 57.08%
	 Val. Loss: 0.810 |  Val. Acc: 63.36% 

	Train Loss: 0.625 | Train Acc: 73.44%
	 Val. Loss: 0.539 |  Val. Acc: 78.24% 

	Train Loss: 0.345 | Train Acc: 87.20%
	 Val. Loss: 0.423 |  Val. Acc: 85.49% 

	Train Loss: 0.191 | Train Acc: 94.34%
	 Val. Loss: 0.341 |  Val. Acc: 90.25% 

	Train Loss: 0.100 | Train Acc: 97.65%
	 Val. Loss: 0.289 |  Val. Acc: 91.96% 

	Train Loss: 0.059 | Train Acc: 98.64%
	 Val. Loss: 0.360 |  Val. Acc: 90.74% 

	Train Loss: 0.033 | Train Acc: 99.29%
	 Val. Loss: 0.288 |  Val. Acc: 93.19% 

	Train Loss: 0.024 | Train Acc: 99.51%
	 Val. Loss: 0.276 |  Val. Acc: 93.42% 

	Train Loss: 0.027 | Train Acc: 99.25%
	 Val. Loss: 0.245 |  Val. Acc: 93.97% 

	Train Loss: 0.018 | Train Acc: 99.58%
	 Val. Loss: 0.239 |  Val. Acc: 94.20% 



In [42]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    #prediction = model(tensor, length_tensor)
    enc = encoders(size_of_vocab, embedding_dim, num_hidden_nodes, num_layers, dropout = dropout).to(device)
    encoder_output = enc(tensor,length_tensor)
    print('Encoder: ',encoder_output)
    dec = decoders(num_hidden_nodes,num_hidden_nodes,10)
    decoder_output = dec(encoder_output)
    print('Decoder: ',decoder_output)

    
    #_, pred = torch.max(prediction, 1) 
    
    #return categories[pred.item()]

In [43]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

Encoder:  tensor([[[ 1.8783e-01,  9.8252e-02, -9.5443e-02, -1.5473e-01,  1.4095e-01,
          -6.8874e-02, -6.1392e-02, -1.3227e-02,  1.0444e-01, -2.3887e-01,
           1.1259e-01,  2.9444e-01, -7.0340e-02,  2.6399e-01,  5.9150e-01,
           1.0275e-01, -2.3289e-01, -9.2818e-02, -9.8889e-02, -2.2673e-02,
           1.3787e-01,  6.3093e-02,  3.8917e-02, -1.7475e-01,  2.8534e-01,
          -4.2722e-02,  2.2544e-01, -1.1018e-01, -6.5911e-02, -2.0856e-02,
           3.3107e-01, -1.8831e-01, -1.1320e-01, -1.4248e-01, -4.6120e-02,
          -3.0054e-01, -3.4589e-02, -2.2964e-01, -2.8125e-01, -2.6467e-01,
           3.6405e-01, -1.3035e-01, -6.9313e-02,  1.8365e-01,  6.0395e-02,
           3.7911e-01,  1.3381e-01, -6.6802e-02, -2.2492e-01,  2.2829e-01,
          -1.6997e-02, -1.3033e-01,  1.2757e-01, -4.3774e-01,  6.2712e-03,
          -1.7449e-01, -1.8719e-02, -7.0177e-02,  1.1963e-01,  1.4875e-01,
           2.6614e-01, -7.9244e-02,  4.6386e-02,  6.7804e-02,  2.8677e-01,
          -1.46

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


IndexError: ignored